# PDEs 3 Workshop 2

Welcome to the second workshop of the PDE 3 (Numerical) course!

## In this workshop:
- Jacobi Iteration
- Successive Over-Relaxation
- Bonus Question: Comparing Analytical and Numerical Solutions

In [1]:
# Run this cell to import the required modules.
# Do this before you write any code!
import numpy as np
import matplotlib.pyplot as plt
from time import process_time
%matplotlib inline  

# Section 1: Jacobi Iteration

In this section, we will use the Jacobi Iteration scheme to solve the Laplace equation for the steady-state heat distribution $u(x,y)$ on a square metal plate which is 30 cm x 30 cm in size.

The boundary conditions of the system are as follows:

* The bottom side is held at $0^\circ C$
* The top side is heated so that the left corner is at $0^\circ C$ and the right corner is at $100^\circ C$, and the temperature varies linearly
* The left side is insulated
* The right side insulated


Mathematically, we can write these as:

* $u(x,0) = 0^\circ C$ 

* $u(x,1) = \frac{x}{30} \times 100^\circ C$

* $\left.\dfrac{\partial u}{\partial x}\right|_{x=0} = 0$

* $\left.\dfrac{\partial u}{\partial x}\right|_{x=1} = 0$

The first two of these boundary conditions are Dirichlet boundary conditions, whereas the last two are Neumann boundary conditions.

### Pseudocode

The steps required for the Jacobi iteration can be written as the following pseudocode.

1. Specify the initial values $u_{i,j}^0$ for all $x_i$,$y_j$
2. Set the iteration counter $m=0$
4. Repeat:

    1. Apply the boundary conditions
    2. Compute $u_{i,j}^{m+1} = \dfrac{1}{2(1+\beta^2)}(u_{i+1,j}^m + u_{i-1,j}^m + \beta^2 (u_{i,j+1} ^m+u_{i,j-1}^m))$ for all $x_i$, $y_j$
    3. Increment $m$
    
5. Stop when $max|u_{i,j}^{m-1}-u_{i,j}^m|<$ tolerance or $m > m_{max}$


## a)

The Jacobi Iteration scheme is based on the second order finite difference approximations
$$
\dfrac{\partial^2u}{\partial x^2} = \dfrac{u_{i-1,j}-2u_{i,j}+u_{i+1,j}}{(\Delta x)^2}
$$

$$
\dfrac{\partial^2u}{\partial y^2} = \dfrac{u_{i,j-1}-2u_{i,j}+u_{i,j+1}}{(\Delta y)^2}
$$

which we can use to determine the Jacobi iteration (see lecture 2 for further details):

$$
u_O = \dfrac{1}{2(1+\beta^2)}(u_E + u_W + \beta^2 (u_N +u_S))
$$
where $\beta=\frac{\Delta x}{\Delta y}$ is the ratio of the mesh spacing in the $x$ and $y$ directions.


<div>
<img src="jacobi.svg" width="500"/>
</div>

Define a function `u_O(u_E, u_W, u_N, u_S, delta_x, delta_y)` below to calculate $u_0$ given the neighbouring values and the desired grid spacing. This is step 3B in our pseudocode.

In [2]:
# Your code here

In [3]:
### BEGIN TESTS ###

assert type(u_O(0, 0, 0, 0, 1, 1)) == float, """Check to ensure that u_O returns a float when only single values are passed into it."""
assert u_O(0, 0, 0, 0, 1, 1) == 0, """Check to ensure that you have implemented u_O correctly using the definition above."""
assert u_O(1, 1, 1, 1, 1, 1) == 1, """Check to ensure that you have implemented u_O correctly using the definition above."""

### END TESTS ###

## b)
Describe the differences between Neumann and Dirichlet boundary conditions and how they can be implemented in the Jacobi Iteration scheme.

<font color='orange'>Your answer goes here. Double-click the cell to modify it.</font>


### Provided for you: The Grid class

To implement Jacobi iteration scheme in code, we will need a grid of points to iterate over. Below is a `Grid` class which we can use. You don't need to know all of the details (though feel free to look through the code if you would like to!) but it has a number of useful methods and variables built in:
- `Grid.x`: The $x$ values of the grid
- `Grid.y`: The $y$ values of the grid
- `Grid.u`: The current values of $u_{i,j}$ on the grid
- `Grid.generate()`: A method (function) to generate the actual grid of points
- `Grid.update()`: A method to update the values of $u_{i,j}$ on the grid
- `Grid.Delta_x()`: A method to calculate $\Delta x$
- `Grid.Delta_y()`: A method to calculate $\Delta y$

In [4]:
class Grid:
    """A class defining a 2D grid on which we can implement the Jacobi and SOR iteration schemes."""

    def __init__(self, ni: int, nj: int):
        self.ni = ni
        self.nj = nj
        self.origin = (0.0, 0.0)
        self.extent = (1.0, 1.0)

        self.u = np.zeros((ni, nj))
        self.x = np.zeros((ni, nj))
        self.y = np.zeros((ni, nj))
    
    def set_origin(self, x0: float, y0: float):
        """Set the origin of the grid."""
        self.origin = (x0, y0)

    def set_extent(self, x1: float, y1: float):
        """Set the extent of the grid."""
        self.extent = (x1, y1)

    def Delta_x(self) -> float:
        """The spacing in the x-direction."""
        return (self.extent[0] - self.origin[0]) / (self.ni - 1)
    
    def Delta_y(self) -> float:
        """The spacing in the y-direction."""
        return (self.extent[1] - self.origin[1]) / (self.nj - 1)
    
    def generate(self, Quiet: bool = True):
        '''generate a uniformly spaced grid covering the domain from the
        origin to the extent.  We are going to do this using linspace from
        numpy to create lists of x and y ordinates and then the meshgrid
        function to turn these into 2D arrays of grid point ordinates.'''
        x_ord = np.linspace(self.origin[0], self.extent[0], self.ni, endpoint=True) # Check whether these should be using endpoint=True
        y_ord = np.linspace(self.origin[1], self.extent[1], self.nj, endpoint=True) # Same here
        self.x, self.y = np.meshgrid(x_ord,y_ord)
        self.x = np.transpose(self.x)
        self.y = np.transpose(self.y)

        if not Quiet:
            print(self)

    def update(self):
        """Update the grid to the new values."""
        # Still need to implement this properly. We don't want to change the boundary condition points so only update the middle points of the grid.
    
    def __str__(self):
        """A quick function to tell us about the grid. This will be what is displayed if you try to print the Grid object."""
        return f"Grid Object: Uniform {self.ni}x{self.nj} grid from {self.origin} to {self.extent}."


### c)

Complete the code snippet below to implement the Jacobi iteration scheme as described in the pseudocode. Remember to account for the Neumann boundary conditions of the system. Referring to our pseudocode, what you write below should capture steps 2-4, calling the function you have written above for step 3B (the `u_0` function). For convenience, the Dirichlet Boundary conditions will be handled separately, as you will see in the function `set_Jacobi_mesh` below.

The function `Jacobi` should take four inputs: the mesh (a ``Grid`` object), the maximum number of iterations, the tolerance criterion and the Neumann boundary condition. The Jacobi iteration stops when the number of iterations exceeds the maximum number specified or if it meets the tolerance criterion. For simplicity, the latter is calculated as the maximum difference in any grid point between the old and new values. Other criteria can be used if desired including integrated quantities. The function should output the number of iterations when the solution converges below the tolerance criterion, and the error at the last iteration.

In [5]:
def Jacobi(mesh: Grid, max_iterations: int, tolerance: float, neumann:float) -> tuple[int, float]:
    """The Jacobi iteration scheme.
    
    Parameters:
    -----------

    mesh: Grid
        The grid on which to implement the Jacobi iteration.
    max_iterations: int
        The maximum number of iterations to perform.
    tolerance: float
        The error tolerance.
    neumann: float
        The Neumann boundary condition.
        Note that in this code, all Neumann boundary conditions are assumed to be the same.
        
    Returns:
    --------
    
    n_iterations: int
        The number of iterations performed.
    error: float
        The error in the solution.
    """

    # Your code here:
    return n_iterations, error

In [6]:
# Now we test the scheme to make sure it works.
### BEGIN TESTS ###
test_grid = Grid(4,4)
test_grid.generate()
Jacobi(test_grid, 3, 1e-3, neumann=0)
assert np.isclose(test_grid.u, np.zeros((4,4))).all(), """Check to ensure that the Jacobi iteration scheme is working correctly. Where the boundary conditions are all zero, nothing should change."""
test_grid.u[0,:] = 1
test_grid.u[-1,:] = 1
test_grid.u[:,0] = 1
test_grid.u[:,-1] = 1
Jacobi(test_grid, 50, 1e-6, neumann=0)
assert np.isclose(test_grid.u, np.ones((4,4))).all(), """Check to ensure that the Jacobi iteration scheme is working correctly. Where the boundary conditions are all one, mesh should converge towards one fairly rapidly."""

Neumann_test_grid = Grid(4,4)
Neumann_test_grid.generate()
Neumann_test_grid.u[:,0] = 1
Neumann_test_grid.u[:,-1] = 1
Jacobi(Neumann_test_grid, 50, 1e-6, neumann=1)
assert np.isclose(Neumann_test_grid.u[:,1], np.array([0.46666348,0.86666428,1.13333095,1.53333015])).all(), """Check to ensure that you have set up the Nuemann boundary conditions correctly."""
### END TESTS ###

### Provided for you: Initialisation of the problem

In the cell below, we define a function `set_Jacobi_mesh` to setup the mesh and implement the Dirichlet boundary conditions given at the start of the section. This essentially covers step 1 of our pseudocode. Try to understand this code, as you will need something similar in future workshops. Run this cell, then move on to the next question. 

In [ ]:
def set_Jacobi_mesh() -> Grid:
    """Set up the Jacobi mesh for the problem."""
    Jacobi_mesh = Grid(101,101)
    Jacobi_mesh.set_extent(30.,30.)
    Jacobi_mesh.set_origin(0., 0.)
    Jacobi_mesh.generate()

    Jacobi_mesh.u[:,0] = 0
    Jacobi_mesh.u[:,-1] = 100/30*Jacobi_mesh.x[:,-1]
    return Jacobi_mesh

Jacobi_mesh = set_Jacobi_mesh()

plt.pcolor(Jacobi_mesh.x, Jacobi_mesh.y, Jacobi_mesh.u)
cax = plt.colorbar()
plt.title("Setup of top and bottom boundary conditions")
plt.xlabel("x")
plt.ylabel("y")
cax.set_label(r"Temperature, $u(x,y)$ / $^\circ$C ")
plt.show()

## d)
Run the Jacobi iteration on the mesh defined above for two cases: one where the solution has not fully converged and one where it has.
Plot the results of both cases on separate axes.
You can create the mesh using the `set_Jacobi_mesh()` function defined above.

[Hint: In the first plot, set ``max_iterations`` to be very low so that this stops the iteration. Play around wih the ``max iterations`` so that you gain some insight into how the solution converges. In the second plot, set ``max_iterations`` to be high enough to allow the iteration to converge.]

In [ ]:
# Your code here.


# Section 2: Successive Over-Relaxation

The Jacobi iteration scheme is relatively simple to implement, but it is not the most efficient iterative scheme.
The Successive Over-Relaxation (SOR) method is a modification of the Jacobi method which can converge faster and is implemented below.
We will use this scheme to solve the Laplace equation for a second system.

In [9]:
def SOR(mesh: Grid, max_iterations: int, tolerance: float) -> tuple[int, float]:
    """The SOR iteration scheme.
    
    Parameters:
    -----------
    
    mesh: Grid
        The grid on which to implement the SOR iteration.
    
    max_iterations: int
        The maximum number of iterations to perform.
    
    tolerance: float
        The error tolerance.
    
    Returns:
    --------
    
    n_iterations: int
        The number of iterations performed.
    
    error: float
        The error in the solution.
    """

    # calculate the optimal value of omega
    lamda = (np.cos(np.pi/mesh.ni)+np.cos(np.pi/mesh.nj))**2/4
    omega = 2/(1+np.sqrt(1-lamda))
    
    # calculate the coefficients
    beta = mesh.Delta_x()/mesh.Delta_y()
    beta_sq = beta**2
    C_beta = 1/(2*(1+beta_sq))
    
    # initialise u_new 
    u_new = mesh.u.copy()

    n_iterations = 0
    
    # itteration
    while n_iterations < max_iterations:
        for i in range(1,mesh.ni-1):
            for j in range(1,mesh.nj-1):
                u_new[i,j] = (1-omega)*mesh.u[i,j] + omega * C_beta*(u_new[i,j-1]+mesh.u[i,j+1]+ beta_sq*(u_new[i-1,j]+mesh.u[i+1,j]))
        
        
        # compute the difference between the new and old solutions
        err = np.max(abs(mesh.u-u_new))
        
        # update the solution
        mesh.u = np.copy(u_new)
        
        # converged?
        if err < tolerance:
            break
        
        n_iterations += 1
    
    return n_iterations, err # return the number of iterations and the final residual

For this part of the workshop, we will solve a Laplace equation

$$
\dfrac{\partial^2 v}{\partial x^2} + \dfrac{\partial^2 v}{\partial y^2} = 0,
$$

with the following boundary conditions:
* $v(0, y) = 0, \quad y\ge0$,
* $v(1, y) = 0, \quad y\ge0$,
* $v(x, y\to\infty) \to 0, \quad 0\le x\le1$,
* $v(x, 0) = \sin^5(\pi x), \quad 0\le x\le1$,

in the region given by $0\le x\le1$ and $y\ge0$.


### a)
Use the `Grid` class to set up a mesh and implement the boundary conditions for this problem.
Produce a plot showing that these boundary conditions have been implemented correctly.
In terms of numerical implementation, it is not possible to implement a boundary condition at $y\to\infty$ directly.
However, this can usually be resolved by taking an upper bound that is sufficiently large for $y$.
For this problem, we can replace $v(x, y\to\infty) \to 0$ with $v(x,3)=0$.

[Hint: You may find the [``plt.pcolor()``](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolor.html) function useful for plotting the grid.]

In [ ]:
# Your code here:


### b) 

Use the SOR implementation above to solve the Laplace equation for this system.

In [ ]:
# Your code here:


## Section 3: Bonus Question

In this workshop, we have used the Jacobi and Successive Over-Relaxation schemes to numerically calculate the solution to the Laplace equation, given a set of boundary conditions.
Now, we will solve the system from Section 2 analytically and compare the analytical and numerical results.

As a reminder, the boundary conditions are:

* $v(0, y) = 0, \quad y\ge0$
* $v(1, y) = 0, \quad y\ge0$
* $v(x, y\to\infty) \to 0, \quad 0\le x\le1$
* $v(x, 0) = \sin^5(\pi x), \quad 0\le x\le1$

### a)
Use the method of separation of variables to solve the Laplace equation subject to the boundary conditions in Section 2.

You might want to use the identity
$$
\sin^5\theta = \frac{1}{16}\left(\sin 5\theta - 5 \sin 3\theta + 10 \sin\theta \right)
$$

<font color='orange'>Your answer goes here. Double-click the cell to modify it.</font>


### b)
Implement the analytic solution you have found in Python and compare the result with the numerical solution from Section 2b.


State why using $y=3$ was a suitable cutoff for the grid in Section 2b.

In [ ]:
# Your code here.



<font color='orange'>Your answer goes here. Double-click the cell to modify it.</font>


### c)
Produce a plot to show how the error between the numerical (using both Jacobi and SOR schemes) and analytic solutions varies with increasing number of iterations. Comment on how quickly the two schemes converge. 

The error can be calculated as:

$$
\varepsilon = \frac{Q_\text{analytic} - Q_\text{numerical}}{Q_\text{analytic}}
$$

where $Q_i$ is defined as 
$$
Q_i = \int_0^1\int_0^{y_\text{max}} u(x,y)^2 \, dx \, dy
$$

To assist with this question, modified versions of the Jacobi and SOR functions above have been provided in ``auxillary.py`` and have been imported below. 
These functions take in the computational mesh (`Grid` object) and the list of iteration steps to be sampled. They output the corresponding iterations and the values of $Q$ for those iterations. An implemenation of the $Q$ integral is also provided. 

In [13]:
# Run this cell to import the required modules.
from auxillary import Jacobi_iteration_vs_error, SOR_iteration_vs_error, calc_Q

In [14]:
# The code below setups the mesh and run the calculations for Jacobi and SOR schemes

mesh_size = (101,301)
mesh_extent = (1., 3.)

iteration_steps = [2**i for i in range(0,11)]

def setup_mesh(mesh_size: tuple[int,int], mesh_extent: tuple[float,float]) -> Grid:
    mesh = Grid(*mesh_size)
    mesh.set_extent(*mesh_extent)
    mesh.generate()
    mesh.u[0,:] = 0
    mesh.u[-1,:] = 0
    mesh.u[:,0] = np.sin(np.pi*mesh.x[:,0])**5
    mesh.u[:,-1] = 0
    return mesh


mesh_Jacobi = setup_mesh(mesh_size, mesh_extent)
iterations_Jacobi, Qs_Jacobi = Jacobi_iteration_vs_error(mesh=mesh_Jacobi, iterations_to_sample=iteration_steps)

mesh_SOR = setup_mesh(mesh_size, mesh_extent)
iterations_SOR, Qs_SOR = SOR_iteration_vs_error(mesh=mesh_SOR, iterations_to_sample=iteration_steps)

mesh_analytic = setup_mesh(mesh_size, mesh_extent)
mesh_analytic.u = analytic(mesh_analytic.x, mesh_analytic.y)
Q_analytic = calc_Q(mesh_analytic)


In [ ]:
# Your code here

